# Parte 1

# ¿Cómo se comparan los datos de ventas de mi empresa en diferentes estados a lo largo del tiempo?

## Introduccion

**Contexto comercial.** Usted es analista de inteligencia comercial para un conglomerado de supermercados. Su empresa tiene tiendas en todo Estados Unidos. La empresa ha recopilado datos que consisten en información de pedidos a nivel de línea de todas sus tiendas. La empresa desea comparar los datos de ventas en diferentes meses y zonas geográficas y poner esta información a disposición de los miembros ejecutivos y accionistas clave. Si cada persona tiene acceso a un tablero interactivo, puede ofrecer una base para un mayor diálogo y una excelente toma de decisiones.

**Problema comercial.** La empresa quiere que **cree un panel interactivo que los ejecutivos de la empresa puedan usar para visualizar las ventas y las ganancias en las ubicaciones de los supermercados durante varios períodos de tiempo.** El panel debe poder usarse a través de un navegador web como Chrome de la empresa.

**Contexto analítico.** En el caso actual, usaremos Dash by Plotly para desarrollar el tablero. Dash es una plataforma de Business Intelligence (BI) de código abierto que nos permite desarrollar paneles interactivos complicados usando solo Python. También nos permite conectarnos a varias fuentes de datos y es fácil de implementar como una aplicación web.

El caso está estructurado de la siguiente manera: (1) aprenderá sobre los componentes básicos de Dash; (2) repasar los elementos básicos para hacer un tablero; y finalmente (3) crear un tablero que compare las ventas y las ganancias a lo largo del tiempo en varios estados.

## Entendiendo a los datos

El punto de partida de nuestro panel de control son siempre nuestros datos. Leámoslo en un DataFrame `pandas` y veamos toda la información que tenemos:

In [1]:
import pandas as pd
df = pd.read_csv('superstore.csv', parse_dates=['Order Date', 'Ship Date'])
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-08-11,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-08-11,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-12-06,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-11-10,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-11-10,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [2]:
df.shape

(9994, 21)

In [3]:
max(df['Order Date'])

Timestamp('2017-12-30 00:00:00')

In [4]:
df.dtypes

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code               int64
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
dtype: object

Los datos de la supertienda tienen datos a nivel de línea para cada producto comprado en diferentes ubicaciones. La lista completa de funciones disponibles se encuentra a continuación:

1. **Order ID**: ID of the order
2. **Order Date**: Date of the order
3. **Ship Date**: Shipping Date of the order
4. **Ship Mode**: Shipping medium
5. **Customer ID**: ID of the customer
6. **Customer Name**: Name of the customer
7. **Segment**: Consumer or corporate Sale
8. **Country**: Country of store
9. **City**: City of store
10. **Postal Code**: Postal code of store
11. **Region**: Region of store
12. **Product ID**: ID of the product
13. **Category**: Category of product
14. **Sub-Category**: Sub-category of product
15. **Product Name**: Name of product
16. **Sales**: Value of sale
17. **Quantity**: Quantity of product sold
18. **Discount**: Discount offered
19. **Profit**: Profit on sale

## Conceptos basicos de Dash

### Setup 

Antes de comenzar a usar Dash, debemos instalar las bibliotecas de Python relevantes que nos permitirán usarlo. Usemos `pip` para instalar las bibliotecas relevantes:

```
pip install dash==1.1.1  # core dash backend
pip install dash-daq==0.1.0  # DAQ componentes
```

Asegurémonos de que todas las extensiones estén instaladas y que la aplicación Dash funcione correctamente. Ejecute el archivo `app_1.py` desde su línea de comando y le dará el enlace HTTP en el que se está ejecutando. Copia y pega este enlace en tu navegador. El código leerá un archivo CSV en un marco de datos `pandas` y trazará un gráfico de barras de "Beneficio por región".

In [5]:
!pip install jupyter_dash
!pip install dash_extensions
!pip install dash_bootstrap_components
!pip install dash_auth

In [6]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go
app = dash.Dash(__name__)
colors = ['red', 'green', 'blue', 'black']
df = pd.read_csv('superstore.csv', parse_dates=['Order Date', 'Ship Date'])
region_grouped = df.groupby(['Region'], as_index=False).sum()

app.layout = html.Div(children=[
    html.H1(children="Profit by Region", style={"textAlign": "center"}),
    dcc.Graph(
            id='regional-profit-plot',
            figure={
                'data': [
                    go.Bar(
                        name='Regional Profit',
                        marker_color=colors,
                        x=region_grouped['Region'],
                        y=region_grouped['Profit']
                    )],
            }
        )
])

C:\Users\Windows\AppData\Local\Temp\ipykernel_67000\2316927919.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\Windows\AppData\Local\Temp\ipykernel_67000\2316927919.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [7]:
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2022 16:56:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:56:40] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_0m1659025711.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:56:40] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_0m1659025711.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:56:40] "GET /_dash-component-suites/dash/deps/react@16.v2_6_0m1659025711.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:56:40] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_0m1659025711.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:56:40] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_0m1659025711.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:56:40] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_6_0m1659025711.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:56:40] "GET /_dash-component-suites/d

In [8]:
try:
    app.run_server(mode='inline', port = 8005, dev_tools_ui=True, debug=False,
              dev_tools_hot_reload =True, threaded=True)
except Exception:
    pass

Dash is running on http://127.0.0.1:8005/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


### Extrayendo fuentes de datos

Las fuentes de datos base generalmente se definen al comienzo de una aplicación Dash. La forma más fácil y común de interactuar con las fuentes de datos de Dash es usar ```pandas```. Todas las funciones de `pandas` también están disponibles para usar dentro de nuestra aplicación Dash:

```
import pandas as pd
df = pd.read_csv(file='file.csv')
```
Si deseas cargar datos de fuentes basadas en SQL, puede usar el comando `pandas.read_sql()`, pasando una consulta SQL y el objeto de conexión SQL SQLAlchemy:
```
import pandas as pd
df = pd.read_sql(sql_command, conn)
```

### Estructura de apps Dash

En esencia, las aplicaciones de Dash son aplicaciones web que consisten en componentes que usan Flask, ReactJS, Javascript y Plotly. Los autores de Dash se han asegurado de que no necesitemos interactuar con ninguno de estos componentes directamente a través de estos otros frameworks y, en cambio, podemos interactuar con todos ellos usando solo Python.

Todas las aplicaciones de Dash se componen de dos partes:
1. **Layout**: Esto consiste en los componentes que usamos en nuestra aplicación (por ejemplo, selectores, gráficos, elementos HTML, div, etc.). El diseño está compuesto por componentes HTML (`dash_html_components`) y componentes Dash (`dash_core_components`). El diseño se establece usando el parámetro `layout` de la variable `app` y es una lista de todos los componentes que aparecerán en el navegador cuando iniciemos nuestra aplicación Dash.

2. **Interactividad**: Este es el filtrado cruzado de datos mediante el cual, al cambiar un gráfico, se modifican otros elementos de nuestro Tablero. Usamos las funciones callbacks de Dash para hacer que nuestra aplicación sea interactiva al consumir entradas de múltiples fuentes y cambiar otros componentes usando la salida. Es algo así como programar con Microsoft Excel: cada vez que cambia una celda de entrada, todas las celdas que dependen de esa celda se actualizarán automáticamente. Esto se llama **programación reactiva**.

## Dash HTML 

Dash nos proporciona componentes HTML que expresamos en Python y que Dash convierte en HTML para el navegador. Por ejemplo:

```
import dash_html_components as html

html.Div([
    html.H1('Hello Dash'),
    html.Div([
        html.P('Dash converts Python classes into HTML'),
        html.P('This conversion happens behind the scenes by Dash')
    ])
])
```

se convertirá en este marcado en el navegador:

```
<div>
    <h1>Hello Dash</h1>
    <div>
        <p>Dash converts Python classes into HTML</p>
        <p>This conversion happens behind the scenes by Dash's JavaScript front-end</p>
    </div>
</div>
```

También podemos pasar varias propiedades a los componentes HTML:

1. **style**: Estos son los mismos que escribiría si estuviera interactuando directamente con las tags HTML. Los nombres de estas etiquetas deben ser camelCased
2. **className**: Los nombres de las clases HTML para estos elementos HTML. Son útiles para interactuar con estos elementos HTML en sus archivos CSS para diseñar
3. **id**: Lo mismo que el ID pasado a los componentes de marcado HTML

De este modo:

```
import dash_html_components as html

html.Div([
    html.Div('Example Div', style={'color': 'blue', 'fontSize': 14}),
    html.P('Example P', className='my-class', id='my-p-element')
], style={'marginBottom': 50, 'marginTop': 25})
```

se representará en el navegador como:


```
<div style="margin-bottom: 50px; margin-top: 25px;">

    <div style="color: blue; font-size: 14px">
        Example Div
    </div>

    <p class="my-class", id="my-p-element">
        Example P
    </p>

</div>
```

Sabiendo esto, echemos un vistazo a `app_2.py` que representa solo un HTML Div con una etiqueta H1 y una etiqueta P. También diseñaremos la etiqueta P para que tenga texto de color azul y estableceremos su ID en `p-tag`. La lista completa de componentes HTML de Dash está disponible aquí: https://dash.plot.ly/dash-html-components.

In [10]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc

app = dash.Dash(__name__)

app.layout = html.Div(children=[ # an html Div
    html.H1(children='Regional Sales Dashboard'), # the first element of the div is an H1, the property children renders the text on the page
    html.P(children="Rendered and Developed using Dash", id="p-tag", style={'color': 'blue'}) # the second element is a P tag
])

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2022 16:57:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:57:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:57:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:57:15] "GET /_favicon.ico?v=2.6.0 HTTP/1.1" 200 -


# Parte 2

## Dash Core Components 

El poder de Dash está anidado dentro de sus componentes principales. Este grupo de componentes nos permite agregar gráficos, controles deslizantes, entradas, tablas, cargadores de archivos y más a nuestra aplicación simplemente usando el código de Python. Cada uno de estos componentes viene con su propio conjunto de parámetros que se pueden configurar para cambiar la apariencia, la sensación y la interactividad de estos componentes. La lista completa de componentes principales de Dash está disponible aquí: https://dash.plot.ly/dash-core-components.

### Graphs y plots

Los gráficos en Dash se representan usando la biblioteca de Plotly y su sintaxis y parámetros son los mismos que los componentes de Plotly. Es decir, el argumento de figura en el componente `dash_core_components.Graph` es el mismo argumento de figura que usa `plotly.py`, etc.

Como ejemplo, veamos `app_3.py` que traza un gráfico de líneas con las ventas y las ganancias de cada mes del año 2015. Tenga en cuenta que el argumento `figure` siempre acepta `data` y `layout` como parámetros. El argumento `datos` es una lista de los gráficos que deben trazarse y el `diseño` tiene parámetros para personalizar la apariencia del gráfico.

In [11]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go


app = dash.Dash(__name__)

df = pd.read_csv('superstore.csv', parse_dates=['Order Date', 'Ship Date'])
df = df[(df['Order Date'] > '2015-01-01') & (df['Order Date'] < '2016-01-01')]
df['month'] = pd.DatetimeIndex(df['Order Date']).month
dff = df.groupby('month', as_index=False).sum()
dff['Total Sales'] = dff['Sales'].cumsum()

app.layout = dcc.Graph(
        id='Data Plot',
        figure={
            'data': [
                go.Scatter(name='Sales', x=dff['month'], y=dff['Sales']),
                go.Scatter(name='Profit', x=dff['month'], y=dff['Profit']),
                go.Scatter(name='Total Sales', x=dff['month'], y=dff['Total Sales'])
            ],
            'layout': {
                'title': 'Sales & Profit in 2015',
                'xaxis': {'title': 'Month'}
            }
        }
    )


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2022 16:57:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:57:28] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:57:28] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:57:28] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:57:28] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


### `help()` command

Dash viene con un comando `help()` que nos permite verificar fácilmente las propiedades de cada uno de estos componentes. Proporciona una lista de todos los parámetros que se pueden pasar al componente. Es muy útil como material de referencia cada vez que nos atascamos con la sintaxis de Dash:

In [32]:
help(dcc.Graph)

Help on class Graph in module dash.dcc.Graph:

class Graph(dash.development.base_component.Component)
 |  Graph(id=undefined, responsive=undefined, clickData=undefined, clickAnnotationData=undefined, hoverData=undefined, clear_on_unhover=undefined, selectedData=undefined, relayoutData=undefined, extendData=undefined, prependData=undefined, restyleData=undefined, figure=undefined, style=undefined, className=undefined, mathjax=undefined, animate=undefined, animation_options=undefined, config=undefined, loading_state=undefined, **kwargs)
 |  
 |  A Graph component.
 |  Graph can be used to render any plotly.js-powered data visualization.
 |  
 |  You can define callbacks based on user interaction with Graphs such as
 |  hovering, clicking or selecting
 |  
 |  Keyword arguments:
 |  
 |  - id (string; optional):
 |      The ID of this component, used to identify dash components in
 |      callbacks. The ID needs to be unique across all of the components
 |      in an app.
 |  
 |  - anima

## Planeando nuestro dashboard

Repasemos ahora el problema empresarial que nos ocupa. Recuerde la solicitud original, que era permitir a los ejecutivos interesados la capacidad de visualizar las ventas y las ganancias en las ubicaciones de los supermercados durante varios períodos de tiempo. ¿Qué características debe tener el tablero para satisfacer esto?

1. La característica de ubicación más predominante en este conjunto de datos es `Estado` (hay un `Código postal`, pero hay tantos códigos postales que esto provocaría que los datos se dividieran tanto que ya no serían fácilmente interpretables) . Por lo tanto, tiene sentido que el tablero permita a los usuarios filtrar las ventas y las ganancias por estado.
2. Podríamos permitir que el usuario ingrese sus propias fechas de inicio y finalización para la visualización. Esto satisface el requisito de que puedan ver las tendencias durante varios períodos de tiempo.

Estos dos puntos dirigen cómo construimos nuestro tablero. Comencemos con el primer punto.

## Análisis a nivel estatal de los datos de ventas

### Configuración de nuestros datos

Agrupemos nuestros datos por estado. Además, dado que la mayoría de las gráficas funcionan con nombres de estados abreviados, usaremos un diccionario para crear una nueva columna llamada `State_abbr` que se derivará de la columna `State`:

In [12]:
us_state_abbrev = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA',
                   'Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','District of Columbia': 'DC',
                   'Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL',
                   'Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA',
                   'Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN',
                   'Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV',
                   'New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY',
                   'North Carolina': 'NC','North Dakota': 'ND','Northern Mariana Islands': 'MP',
                   'Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Palau': 'PW','Pennsylvania': 'PA',
                   'Puerto Rico': 'PR','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD',
                   'Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virgin Islands': 'VI',
                   'Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI',
                   'Wyoming': 'WY',}
df = pd.read_csv('superstore.csv', parse_dates=['Order Date'])
df['State_abbr'] = df['State'].map(us_state_abbrev)
states_grouped = df.groupby(['State_abbr'], as_index=False).sum()
states_grouped.head()

,State_abbr,Row ID,Postal Code,Sales,Quantity,Discount,Profit
0,AL,243876,2195669,19510.6400,256,0.0,5786.8253
1,AR,304575,4339309,11678.1300,240,0.0,4008.6871
2,AZ,1019876,19102126,35282.0010,862,68.0,-3427.9246
3,CA,10137449,184382639,457687.6315,7667,145.6,76381.3871
4,CO,797002,14613828,32108.1180,693,57.6,-6527.8579


### Definiendo el layout 

El diseño es la parte más importante de nuestra parcela. Es lo que nos ayuda a que nuestras tramas destaquen y llamen la atención del usuario.

Comencemos agregando un encabezado en una etiqueta `div` que será el título de nuestro tablero:

```
# app_4.py
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])

app.layout = html.Div(children=[
    html.H2(children="US Sales Map"),
])

if __name__ == "__main__":
    app.run_server(debug=True)
```

In [13]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go

app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])

app.layout = html.Div(children=[
    html.H2(children="US Sales Map"),
])

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2022 16:57:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:57:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:57:53] "GET /_dash-dependencies HTTP/1.1" 200 -


### Añadiendo HTML & CSS 

Agreguemos una clase CSS al texto colocándolo en un div y configurando la clase del div principal en `study-browser-banner`. Los estilos para estas clases se definen en la hoja de estilo externa:


```
# app_5.py
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])

app.layout = html.Div(children=[
    html.Div(
        children=[html.H2(children="US Sales Dashboard", className='h2-title'),],
        className='study-browser-banner row'
    )
])

if __name__ == "__main__":
    app.run_server(debug=True)
```

In [14]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go


app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])

app.layout = html.Div(children=[
    html.Div(
        children=[html.H2(children="US Sales Dashboard", className='h2-title'),],
        className='study-browser-banner row'
    )
])

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2022 16:58:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:58:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:58:05] "GET /_dash-dependencies HTTP/1.1" 200 -


### Añadiendo mapa al dashboard

Ahora, agreguemos nuestro mapa al diseño. Usaremos el componente Mapbox de Plotly.

Para usar Mapbox, necesitaremos obtener una clave gratuita del sitio web de Mapbox. Este token de acceso nos permite dibujar e interactuar con los mapas de Mapbox con Plotly. El archivo Mapbox también nos permitirá usar un archivo geo-JSON que tendrá información sobre los estados de EE. UU. Configuremos el token de acceso y almacenemos la estructura geo-JSON en una variable:

In [15]:
import json
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
with open('us.json') as f:
    geojson = json.loads(f.read())

Agreguemos nuestro mapa al cuerpo de nuestra página creando un nuevo HTML Div y configurando su clase en `row app-body`.

La clase Plotly para el gráfico que vamos a usar para esto se llama `ChoroplethMapBox`. El parámetro `locations` se refiere a las ubicaciones que trazaremos y el parámetro `z` tiene los valores que se trazarán en el mapa. Usamos una escala de colores que colorea los estados según la cantidad de ventas en ese estado, como un mapa de calor.

El diseño de este gráfico se configurará a través de la propiedad `layout` de la figura, donde configuramos nuestro token de acceso a Mapbox. Echemos un vistazo a `app_6.py` que pone todo esto junto.

In [16]:
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])

app.layout = html.Div(children=[
    html.Div(
        children=[html.H2(children="US Sales Dashboard", className='h2-title'),],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body', 
        children=[
        dcc.Graph(
            id='map-plot',
            figure={ 
                'data': [go.Choroplethmapbox(
                    geojson=geojson,
                    locations=states_grouped['State_abbr'],
                    z=states_grouped['Sales'],
                    colorscale='Viridis',
                    colorbar_title="Thousands USD"
                )],
                'layout': go.Layout(
                        mapbox_style="dark",
                        mapbox_accesstoken=token,
                        mapbox_zoom=2,
                        mapbox_center = {"lat": 37.0902, "lon": -95.7129}
                    )
            }
        )
])])


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2022 16:58:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:58:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:58:23] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:58:23] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2022 16:58:23] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


### Ejercicio 1

Establezca un texto flotante en el mapa que muestre el nombre del estado y su nivel de ventas en formato de dos puntos decimales.

In [38]:
df = pd.read_csv('superstore.csv', parse_dates=['Order Date'])
df['State_abbr'] = df['State'].map(us_state_abbrev)
states_grouped = df.groupby(['State_abbr'], as_index=False).sum()
states_grouped['Sales_State'] = states_grouped[['State_abbr', 'Sales']].apply(lambda x: 'Sales for {}: {:.2f}$'.format(x[0], x[1]), axis=1)
states_grouped.head()

token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
with open('us.json') as f:
    geojson = json.loads(f.read())

app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])

app.layout = html.Div(children=[
    html.Div(
        children=[html.H2(children="US Sales Dashboard", className='h2-title'),],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body', 
        children=[
        dcc.Graph(
            id='map-plot',
            figure={ 
                'data': [go.Choroplethmapbox(
                    geojson=geojson,
                    locations=states_grouped['State_abbr'],
                    z=states_grouped['Sales'],
                    colorscale='Viridis',
                    text=states_grouped['Sales_State'],
                    colorbar_title="Thousands USD"
                )],
                'layout': go.Layout(
                        mapbox_style="dark",
                        mapbox_accesstoken=token,
                        mapbox_zoom=2,
                        mapbox_center = {"lat": 37.0902, "lon": -95.7129}
                    )
            }
        )
])])


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# Parte 3

## Períodos de tiempo personalizados para datos de ventas

Ahora, agreguemos una función que permita a los usuarios seleccionar un período de tiempo personalizado para ver los datos de ventas. A través de esto, aprenderemos el concepto de **callbacks** en Dash.

### Agergando un DateTime selector 

Ahora agregaremos un selector de fecha y hora a nuestro tablero. Trabajando con nuestro diseño existente, agreguemos un componente de rango de selección de fechas a nuestro diseño. Vamos a crear un nuevo Div en el lado derecho de nuestro encabezado. Nuevamente, nuestro CSS ya tiene clases integradas para esto. Solo necesitamos agregar el componente Dash en un HTML Div y establecer sus clases.

Las fechas `start_date` y `end_date` en el selector se establecerán de forma predeterminada en las fechas más tempranas y más recientes de nuestro conjunto de datos. `app_8.py` muestra cómo funciona todo esto.

In [39]:
df = pd.read_csv('superstore.csv', parse_dates=['Order Date'])
df['State_abbr'] = df['State'].map(us_state_abbrev)
states_grouped = df.groupby(['State_abbr'], as_index=False).sum()
states_grouped['Sales_State'] = states_grouped[['State_abbr', 'Sales']].apply(lambda x: 'Sales for {}: {:.2f}$'.format(x[0], x[1]), axis=1)

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="US Sales Dashboard", className='h2-title'),
            html.Div(
                className='div-logo padding-top-bot',
                children=[
                    dcc.DatePickerRange(
                        id='date-picker-range', # The id of the DatePicker, its always very important to set an Id for all our components
                        start_date=df['Order Date'].min(), # The start_date is going to be the min of Order Date in our dataset
                        end_date=df['Order Date'].max(),
                    )
                ]
            )],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body', 
        children=[
            html.Div(
                className='twelve columns',
                children=[
                    html.Div(
                        dcc.Graph(
                    id='map-plot',
                    figure={ 
                        'data': [go.Choroplethmapbox(
                            geojson=geojson,
                            locations=states_grouped['State_abbr'],
                            z=states_grouped['Sales'],
                            colorscale='Viridis',
                            text=states_grouped['Sales_State'],
                            colorbar_title="Thousands USD"
                        )],
                        'layout': go.Layout(
                                mapbox_style="dark",
                                mapbox_accesstoken=token,
                                mapbox_zoom=3,
                                margin={'t': 10, 'l': 10, 'r': 10, 'b': 10},
                                mapbox_center = {"lat": 37.0902, "lon": -95.7129}
                            )
                    }
                ) 
                    )
                ]
            )
        
])])


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


### Conexión del selector DateTime al mapa mediante callbacks (10 mts)

Todas las funciones callbacks necesitan al menos una entrada, que se especifica mediante `dash.dependencies.Input()`. Las entradas se pasan a la función como parámetros que luego podemos usar en nuestra función. Podemos hacer referencia a los componentes de nuestro diseño en nuestra función de devolución de llamada usando sus `id`s.

Actualmente, las funciones callback solo admiten una salida a través de `dash.dependencies.Output()`. El primer parámetro de esto es el `id` del componente que necesitamos actualizar y el segundo parámetro es el argumento que vamos a cambiar en ese componente.

Hagamos una función callback que tomará la entrada del rango del selector de fechas y actualizará nuestro Cloropeth para reflejar las ventas durante el período seleccionado. Ahora, cada vez que se actualice el rango de fechas, nuestra trama cambiará:


In [ ]:
# NOTE: No ejecutar esta celda solo es para motivos de explicaicon
@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component con el id map-plot 
    [
        dash.dependencies.Input('date-picker-range', 'start_date'), 
        dash.dependencies.Input('date-picker-range', 'end_date')
    ]
)
def update_sales_map(start_date, end_date):
    dff = df[(df['Order Date'] >= start_date) & (df['Order Date'] < end_date)] 
    states_grouped = dff.groupby(['State_abbr'], as_index=False).sum()  # Agrupacion
    states_grouped['Sales_State'] = states_grouped[['State_abbr', 'Sales']].apply(lambda x: 'Sales for {}: {:.2f}$'.format(x[0], x[1]), axis=1)

    return { 
            'data': [go.Choroplethmapbox(
                geojson=geojson,
                locations=states_grouped['State_abbr'],
                z=states_grouped['Sales'],
                colorscale='Viridis',
                text=states_grouped['Sales_State'],
                colorbar_title="Thousands USD"
            )],
            'layout': go.Layout(
                    mapbox_style="dark",
                    mapbox_accesstoken=token,
                    mapbox_zoom=3,
                    margin={'t': 10, 'l': 10, 'r': 10, 'b': 10},
                    mapbox_center = {"lat": 37.0902, "lon": -95.7129}
                )
        }

### Poniendo todo junto

Pulling everything together gives us the code in `app_9.py`.

In [19]:
df = pd.read_csv('superstore.csv', parse_dates=['Order Date'])
df['State_abbr'] = df['State'].map(us_state_abbrev)
states_grouped = df.groupby(['State_abbr'], as_index=False).sum()
states_grouped['Sales_State'] = states_grouped[['State_abbr', 'Sales']].apply(lambda x: 'Sales for {}: {:.2f}$'.format(x[0], x[1]), axis=1)

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="US Sales Dashboard", className='h2-title'),
            html.Div(
                className='div-logo padding-top-bot',
                children=[
                    dcc.DatePickerRange(
                        id='date-picker-range', # The id of the DatePicker, its always very important to set an Id for all our components
                        start_date=df['Order Date'].min(), # The start_date is going to be the min of Order Date in our dataset
                        end_date=df['Order Date'].max(),
                    )
                ]
            )],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body', 
        children=[
            html.Div(
                className='twelve columns',
                children=[
                    html.Div(
                        dcc.Graph(
                    id='map-plot',
                    figure={ 
                        'data': [go.Choroplethmapbox(
                            geojson=geojson,
                            locations=states_grouped['State_abbr'],
                            z=states_grouped['Sales'],
                            colorscale='Viridis',
                            text=states_grouped['Sales_State'],
                            colorbar_title="Thousands USD"
                        )],
                        'layout': go.Layout(
                                mapbox_style="dark",
                                mapbox_accesstoken=token,
                                mapbox_zoom=3,
                                margin={'t': 10, 'l': 10, 'r': 10, 'b': 10},
                                mapbox_center = {"lat": 37.0902, "lon": -95.7129}
                            )
                    }
                ) 
                    )
                ]
            )
        
])])

@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component with id map-plot will be changed, the 'figure' argument is updated
    [
        dash.dependencies.Input('date-picker-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-picker-range', 'end_date')
    ]
)
def update_sales_map(start_date, end_date):
    dff = df[(df['Order Date'] >= start_date) & (df['Order Date'] < end_date)] # We filter our dataset for the daterange
    states_grouped = dff.groupby(['State_abbr'], as_index=False).sum()  # We group our data again
    states_grouped['Sales_State'] = states_grouped[['State_abbr', 'Sales']].apply(lambda x: 'Sales for {}: {:.2f}$'.format(x[0], x[1]), axis=1)

    return { 
            'data': [go.Choroplethmapbox(
                geojson=geojson,
                locations=states_grouped['State_abbr'],
                z=states_grouped['Sales'],
                colorscale='Viridis',
                text=states_grouped['Sales_State'],
                colorbar_title="Thousands USD"
            )],
            'layout': go.Layout(
                    mapbox_style="dark",
                    mapbox_accesstoken=token,
                    mapbox_zoom=3,
                    margin={'t': 10, 'l': 10, 'r': 10, 'b': 10},
                    mapbox_center = {"lat": 37.0902, "lon": -95.7129}
                )
        }



if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Si cambiamos el rango de fechas, nuestro mapa también se actualizará. Todos los tableros son combinaciones de componentes y devoluciones de llamada con HTML y CSS incorporados para que nuestro tablero se vea más bonito y más útil. Puede ver una gran cantidad de paneles de muestra aquí: https://dash-gallery.plotly.host/Portal/.

### Ejercicio 2:

Ahora tenemos un mapa completamente funcional que se actualiza a medida que cambia el período seleccionado. Intentemos responder las siguientes preguntas comerciales:

1. ¿Qué estado tuvo las ventas más altas en diciembre de 2015?
2. ¿Qué estado tuvo las ventas más bajas en el año 2015?

**Responder**. Montana tuvo las ventas más altas en diciembre de 2015 y Maine tuvo las ventas más bajas para el año de 2015.

## Iteración basada en comentarios ejecutivos

Después de presentar el trabajo anterior a los ejecutivos, indican que, en general, están contentos con él. Sin embargo, mencionan que también les gustaría profundizar en cada estado y ver las ventas y las ganancias de las diferentes categorías en las regiones seleccionadas:

1. A la empresa le gustaría ver las ventas de todas las categorías en nuestros datos durante un período de tiempo seleccionado. Por lo tanto, debemos poder graficar tendencias de ventas para múltiples categorías en el mismo gráfico. Un gráfico de líneas, con el mes y el año en el eje x y las ventas en el eje y, nos permite hacer esto.
2. La empresa quiere ver las ventas frente a las ganancias cada año para varias categorías en las regiones seleccionadas del mapa. Por lo tanto, debemos poder ver las categorías una al lado de la otra, así como comparar fácilmente las ventas y las ganancias. Un gráfico de barras agrupadas, agrupadas por año en el eje x y mostrando las ventas y las ganancias en el eje y, funciona bien para esto.

Necesitamos agrupar y agregar elementos de línea de ventas individuales para estos gráficos, lo que significa que primero debemos analizar las cantidades de mes y año del campo "Fecha de pedido". Agreguemos una columna `YearMonth` a nuestro DataFrame como un campo `datetime`:

In [20]:
df['YearMonth'] = pd.to_datetime(df['Order Date'].map(lambda x: "{}-{}".format(x.year, x.month)))

### Trazar un gráfico de líneas con varias líneas

Para trazar varias líneas en el mismo gráfico, creamos una matriz de objetos Plotly Graph y los configuramos en el atributo `figure` de nuestro objeto `dcc.Graph` en el diseño. A continuación, todas las líneas se representan en el mismo gráfico. Sin embargo, debido a que Plotly no tiene un objeto Graph de gráfico de líneas directo, usaremos `go.Scatter()` de Plotly para crear una serie de objetos gráficos de diagramas de dispersión, que luego podemos conectar usando segmentos de línea para imitar un gráfico de líneas. Cada objeto de esta serie corresponde a una sola categoría a lo largo del tiempo:

In [21]:
def category_month_scatter_plot(df, states=None, start_date=None, end_date=None):
    data = []
    
    subcategory_grouped = df.groupby(['Category'], as_index=False)
    for name, group in subcategory_grouped:
        grouped = group.groupby('YearMonth', as_index=False).sum()
        data.append(go.Scatter(
            x=grouped['YearMonth'], y=grouped['Sales'], name=name
        ))
    
    return data

Let's now add this to our layout, as shown in `app_10.py`.

In [22]:
df = pd.read_csv('superstore.csv', parse_dates=['Order Date'])
df['YearMonth'] = pd.to_datetime(df['Order Date'].map(lambda x: "{}-{}".format(x.year, x.month)))
df['State_abbr'] = df['State'].map(us_state_abbrev)
states_grouped = df.groupby(['State_abbr'], as_index=False).sum()
states_grouped['Sales_State'] = states_grouped[['State_abbr', 'Sales']].apply(lambda x: 'Sales for {}: {:.2f}$'.format(x[0], x[1]), axis=1)

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="US Sales Dashboard", className='h2-title'),
            html.Div(
                className='div-logo padding-top-bot',
                children=[
                    dcc.DatePickerRange(
                    id='date-picker-range', # The id of the DatePicker, its always very important to set an Id for all our components
                    start_date=df['Order Date'].min(), # The start_date is going to be the min of Order Date in our dataset
                    end_date=df['Order Date'].max(),
                )
                ]
            )
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body', 
        children=[
            html.Div(
                className='twelve columns card-left',
                children=[
                    html.Div(
                        dcc.Graph(
                            id='map-plot',
                            figure={ 
                                'data': [go.Choroplethmapbox(
                                    geojson=geojson,
                                    locations=states_grouped['State_abbr'],
                                    z=states_grouped['Sales'],
                                    colorscale='Viridis',
                                    text=states_grouped['Sales_State'],
                                    colorbar_title="Thousands USD"
                                )],
                                'layout': go.Layout(
                                        mapbox_style="dark",
                                        mapbox_accesstoken=token,
                                        mapbox_zoom=3,
                                        margin={'t': 0, 'l': 0, 'r': 0, 'b': 0},
                                        mapbox_center={"lat": 37.0902, "lon": -95.7129}
                                    )
                            }
                        ),
                    ),
                ]),
            html.Div( # New Div
                className='twelve columns',
                children=[
                    html.Div(
                        className='six columns',
                        children=[dcc.Graph(
                            id='category-scatter-plot', # Plot 1
                            figure={
                                'data': category_month_scatter_plot(df),
                                'layout': go.Layout(
                                    margin={'t': 10, 'r': 10,}
                                )
                            },
                        )]
                    ),
                    html.Div(
                        className='six columns',
                        children=[dcc.Graph(id='category-bar-plot')]
                    ),
                ]
            )]
        )]
)


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


### Pregunta:

Ahora puede seleccionar una categoría particular de la leyenda de la derecha. A ver si puedes responder a las siguientes preguntas:

1. La categoría Muebles presenta estacionalidad en diciembre, cuando sus ventas son las más altas. Solo hay un año durante el cual los muebles no se venden tan bien durante ese mes. ¿Qué año es?
2. ¿Qué categoría ha experimentado una tendencia alcista en las ventas año tras año?

### Ejercicio 3

Agregue un gráfico de barras agrupadas, agrupadas por año, donde cada barra muestre las ventas y las ganancias de una categoría en particular.

Para hacer un gráfico de barras agrupadas, agregamos las gráficas a una lista de Python (consulte la función `category_bar_plot()`). Para cada gráfico, inicializamos una matriz `data` a la que agregamos dos gráficos `go.Bar()` (uno para ventas y otro para ganancias).
El eje x de la gráfica debe ser la categoría y el eje y debe ser las ventas y las ganancias de esa categoría.

El valor de retorno de esta función se pasa al atributo `figure` de nuestro diagrama original.

In [24]:
def category_bar_plot(df, states=None, start_date=None, end_date=None):
    data = []
    category_grouped = df.groupby(['Category'], as_index=False).sum()
    data.append(go.Bar(
        x=category_grouped['Category'], y=category_grouped['Sales'], name='Sales'
    ))
    data.append(go.Bar(
        x=category_grouped['Category'], y=category_grouped['Profit'], name='Profit'
    ))
    return data

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="US Sales Dashboard", className='h2-title'),
            html.Div(
                className='div-logo padding-top-bot',
                children=[
                    dcc.DatePickerRange(
                    id='date-picker-range', # The id of the DatePicker, its always very important to set an Id for all our components
                    start_date=df['Order Date'].min(), # The start_date is going to be the min of Order Date in our dataset
                    end_date=df['Order Date'].max(),
                )
                ]
            )
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body', 
        children=[
            html.Div(
                className='twelve columns card-left',
                children=[
                    html.Div(
                        dcc.Graph(
                            id='map-plot',
                            figure={ 
                                'data': [go.Choroplethmapbox(
                                    geojson=geojson,
                                    locations=states_grouped['State_abbr'],
                                    z=states_grouped['Sales'],
                                    colorscale='Viridis',
                                    text=states_grouped['Sales_State'],
                                    colorbar_title="Thousands USD"
                                )],
                                'layout': go.Layout(
                                        mapbox_style="dark",
                                        mapbox_accesstoken=token,
                                        mapbox_zoom=3,
                                        margin={'t': 0, 'l': 0, 'r': 0, 'b': 0},
                                        mapbox_center={"lat": 37.0902, "lon": -95.7129}
                                    )
                            }
                        ),
                    ),
                ]),
            html.Div( # New Div
                className='twelve columns',
                children=[
                    html.Div(
                        className='six columns',
                        children=[dcc.Graph(
                            id='category-scatter-plot', # Plot 1
                            figure={
                                'data': category_month_scatter_plot(df),
                                'layout': go.Layout(
                                    margin={'t': 10, 'r': 10,}
                                )
                            },
                        )]
                    ),
                    html.Div(
                        className='six columns',
                        children=[dcc.Graph(id='category-bar-plot', figure={'data': category_bar_plot(df), 'layout':{'barmode': 'group'}})]
                    ),
                ]
            )]
        )]
)


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


This shows that Furniture has the worst sales-to-profit ratio, whereas Technology has the best one.

## Actualización  al seleccionar el mapa

Ahora agregaremos una función de devolución de llamada que actualiza el gráfico de líneas cada vez que hacemos una selección en el mapa. Hagamos que esta función de devolución de llamada tome como entradas la selección del mapa y los filtros de fecha. La propiedad `selectedData` del mapa contiene las regiones seleccionadas. Tenga en cuenta que TODOS los gráficos tienen un atributo `selectedData` en sus entradas:

In [25]:
# No correr esto es solo para explciar
@app.callback(
    dash.dependencies.Output('category-scatter-plot', 'figure'),
    [
        dash.dependencies.Input('map-plot', 'selectedData'),
        dash.dependencies.Input('date-picker-range', 'start_date'),
        dash.dependencies.Input('date-picker-range', 'end_date')
    ]
)
def update_scatter_plot(map_data, start_date, end_date):
    print(map_data)

### Lectura de datos de entrada

Una forma útil de verificar los valores que vienen en los parámetros de entrada es agregar una declaración de impresión simple a nuestra función. Para los datos del mapa, así es como se ve la entrada:

```
{'puntos': [{'curveNumber': 0, 'pointNumber': 41, 'pointIndex': 41, 'ubicación': 'TX', 'z': 170188.04580000005, 'text': 'Ventas para TX: 170188.05' }]}
```

A partir de esto, vemos que `puntos` es una lista de objetos de diccionario con todos los estados seleccionados. Luego podemos extraer la `ubicación` de cada uno. Completaremos nuestra función de devolución de llamada en dos pasos:

1. Actualice `category_month_scatter_plot()` para filtrar datos según los argumentos de entrada de los estados seleccionados, la fecha de inicio y la fecha de finalización
2. Pase los valores actualizados a la figura de `category-scatter-plot`

El resultado se muestra en `app_12.py`.

In [26]:
def category_month_scatter_plot(df, states=None, start_date=None, end_date=None):
    data = []
    dff = df.copy()
    if states:
        dff = dff[(df['State_abbr'].isin(states))]
        
    if  start_date and end_date:
        dff = dff[(df['Order Date'] >= start_date) & (df['Order Date'] <= end_date)]
   
    subcategory_grouped = dff.groupby(['Category'], as_index=False)
    for name, group in subcategory_grouped:
        grouped = group.groupby('YearMonth', as_index=False).sum()
        data.append(go.Scatter(
            x=grouped['YearMonth'], y=grouped['Sales'], name=name
        ))
    
    return data

def category_bar_plot(df, states=None, start_date=None, end_date=None):
    data = []
    category_grouped = df.groupby(['Category'], as_index=False).sum()
    data.append(go.Bar(
        x=category_grouped['Category'], y=category_grouped['Sales'], name='Sales'
    ))
    data.append(go.Bar(
        x=category_grouped['Category'], y=category_grouped['Profit'], name='Profit'
    ))
    return data


app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
with open('us.json') as f:
    geojson = json.loads(f.read())

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands': 'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
df = pd.read_csv('superstore.csv', parse_dates=['Order Date'])
df['YearMonth'] = pd.to_datetime(df['Order Date'].map(lambda x: "{}-{}".format(x.year, x.month)))
df['State_abbr'] = df['State'].map(us_state_abbrev)
states_grouped = df.groupby(['State_abbr'], as_index=False).sum()
states_grouped['Sales_State'] = states_grouped[['State_abbr', 'Sales']].apply(lambda x: 'Sales for {}: {:.2f}$'.format(x[0], x[1]), axis=1)


app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="US Sales Dashboard", className='h2-title'),
            html.Div(
                className='div-logo padding-top-bot',
                children=[
                    dcc.DatePickerRange(
                    id='date-picker-range', # The id of the DatePicker, its always very important to set an Id for all our components
                    start_date=df['Order Date'].min(), # The start_date is going to be the min of Order Date in our dataset
                    end_date=df['Order Date'].max(),
                )
                ]
            )
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body', 
        children=[
            html.Div(
                className='twelve columns card-left',
                children=[
                    html.Div(
                        dcc.Graph(
                            id='map-plot',
                            figure={ 
                                'data': [go.Choroplethmapbox(
                                    geojson=geojson,
                                    locations=states_grouped['State_abbr'],
                                    z=states_grouped['Sales'],
                                    colorscale='Viridis',
                                    text=states_grouped['Sales_State'],
                                    colorbar_title="Thousands USD"
                                )],
                                'layout': go.Layout(
                                        mapbox_style="dark",
                                        mapbox_accesstoken=token,
                                        mapbox_zoom=3,
                                        margin={'t': 0, 'l': 0, 'r': 0, 'b': 0},
                                        mapbox_center={"lat": 37.0902, "lon": -95.7129}
                                    )
                            }
                        ),
                    ),
                ]),
            html.Div( # New Div
                className='twelve columns',
                children=[
                    html.Div(
                        className='six columns',
                        children=[dcc.Graph(
                            id='category-scatter-plot', # Plot 1
                            figure={
                                'data': category_month_scatter_plot(df),
                                'layout': go.Layout(
                                    margin={'t': 10, 'r': 10,}
                                )
                            },
                        )]
                    ),
                    html.Div(
                        className='six columns',
                        children=[dcc.Graph(id='category-bar-plot', figure={'data': category_bar_plot(df), 'layout':{'barmode': 'group'}})]
                    ),
                ]
            )]
        )]
)


@app.callback(
    dash.dependencies.Output('category-scatter-plot', 'figure'),
    [
        dash.dependencies.Input('map-plot', 'selectedData'),
        dash.dependencies.Input('date-picker-range', 'start_date'),
        dash.dependencies.Input('date-picker-range', 'end_date')
    ]
)


def update_scatter_plot(map_data, start_date, end_date):
    states = [each['location'] for each in map_data['points']] if map_data else None  # extract states
    data = category_month_scatter_plot(df, states, start_date, end_date)
    return {
        'data': data,
        'layout': go.Layout(
            margin={'t': 10, 'r': 10,}
        )
    }

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


### Pregunta:

Ahora podemos jugar con el mapa y las fechas de inicio y finalización para profundizar en las categorías. Vea si puede encontrar las respuestas a las siguientes preguntas de nuestras parcelas:

1. ¿Cuál es la categoría de mayor venta en California en diciembre de 2015?
2. ¿Cuál es la categoría de menor venta en Texas en marzo de 2016?

### Ejercicio 4

Agregue otra función callback que actualice nuestro gráfico de barras con capacidades de selección de estado y fecha. Reúna todas las funciones de devolución de llamada del tablero en un programa para que podamos llegar a una salida final de la aplicación.

In [27]:
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])


def category_month_scatter_plot(df, states=None, start_date=None, end_date=None):
    data = []
    dff = df.copy()
    if states:
        dff = dff[(df['State_abbr'].isin(states))]
        
    if  start_date and end_date:
        dff = dff[(df['Order Date'] >= start_date) & (df['Order Date'] <= end_date)]

    subcategory_grouped = dff.groupby(['Category'], as_index=False)
    for name, group in subcategory_grouped:
        grouped = group.groupby('YearMonth', as_index=False).sum()
        data.append(go.Scatter(
            x=grouped['YearMonth'], y=grouped['Sales'], name=name
        ))
    
    return data


def category_bar_plot(df, states=None, start_date=None, end_date=None):
    data = []
    
    dff = df.copy()
    if states:
        dff = dff[(df['State_abbr'].isin(states))]
        
    if  start_date and end_date:
        dff = dff[(df['Order Date'] >= start_date) & (df['Order Date'] <= end_date)]

    category_grouped = dff.groupby(['Category'], as_index=False).sum()
    data.append(go.Bar(
        x=category_grouped['Category'], y=category_grouped['Sales'], name='Sales'
    ))
    data.append(go.Bar(
        x=category_grouped['Category'], y=category_grouped['Profit'], name='Profit'
    ))
    return data


app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
with open('us.json') as f:
    geojson = json.loads(f.read())

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands': 'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
df = pd.read_csv('superstore.csv', parse_dates=['Order Date'])
df['YearMonth'] = pd.to_datetime(df['Order Date'].map(lambda x: "{}-{}".format(x.year, x.month)))
df['State_abbr'] = df['State'].map(us_state_abbrev)
states_grouped = df.groupby(['State_abbr'], as_index=False).sum()
states_grouped['Sales_State'] = states_grouped[['State_abbr', 'Sales']].apply(lambda x: 'Sales for {}: {:.2f}$'.format(x[0], x[1]), axis=1)


app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="US Sales Dashboard", className='h2-title'),
            html.Div(
                className='div-logo padding-top-bot',
                children=[
                    dcc.DatePickerRange(
                    id='date-picker-range', # The id of the DatePicker, its always very important to set an Id for all our components
                    start_date=df['Order Date'].min(), # The start_date is going to be the min of Order Date in our dataset
                    end_date=df['Order Date'].max(),
                )
                ]
            )
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body', 
        children=[
            html.Div(
                className='twelve columns card-left',
                children=[
                    html.Div(
                        dcc.Graph(
                            id='map-plot',
                            figure={ 
                                'data': [go.Choroplethmapbox(
                                    geojson=geojson,
                                    locations=states_grouped['State_abbr'],
                                    z=states_grouped['Sales'],
                                    colorscale='Viridis',
                                    text=states_grouped['Sales_State'],
                                    colorbar_title="Thousands USD"
                                )],
                                'layout': go.Layout(
                                        mapbox_style="dark",
                                        mapbox_accesstoken=token,
                                        mapbox_zoom=3,
                                        margin={'t': 0, 'l': 0, 'r': 0, 'b': 0},
                                        mapbox_center={"lat": 37.0902, "lon": -95.7129}
                                    )
                            }
                        ),
                    ),
                ]),
            html.Div( # New Div
                className='twelve columns',
                children=[
                    html.Div(
                        className='six columns',
                        children=[dcc.Graph(
                            id='category-scatter-plot', # Plot 1
                            figure={
                                'data': category_month_scatter_plot(df),
                                'layout': go.Layout(
                                    margin={'t': 10, 'r': 10,}
                                )
                            },
                        )]
                    ),
                    html.Div(
                        className='six columns',
                        children=[dcc.Graph(id='category-bar-plot', figure={'data': category_bar_plot(df), 'layout':{'barmode': 'group'}})]
                    ),
                ]
            )]
        )]
)


@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component with id map-plot will be changed, the 'figure' argument is updated
    [
        dash.dependencies.Input('date-picker-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-picker-range', 'end_date')
    ]
)
def update_sales_map(start_date, end_date):
    dff = df[(df['Order Date'] >= start_date) & (df['Order Date'] < end_date)] # We filter our dataset for the daterange
    states_grouped = dff.groupby(['State_abbr'], as_index=False).sum()  # We group our data again
    states_grouped['Sales_State'] = states_grouped[['State_abbr', 'Sales']].apply(lambda x: 'Sales for {}: {:.2f}$'.format(x[0], x[1]), axis=1)

    return { 
            'data': [go.Choroplethmapbox(
                geojson=geojson,
                locations=states_grouped['State_abbr'],
                z=states_grouped['Sales'],
                colorscale='Viridis',
                text=states_grouped['Sales_State'],
                colorbar_title="Thousands USD"
            )],
            'layout': go.Layout(
                    mapbox_style="dark",
                    mapbox_accesstoken=token,
                    mapbox_zoom=3,
                    margin={'t': 10, 'l': 10, 'r': 10, 'b': 10},
                    mapbox_center = {"lat": 37.0902, "lon": -95.7129}
                )
        }


@app.callback(
    dash.dependencies.Output('category-scatter-plot', 'figure'),
    [
        dash.dependencies.Input('map-plot', 'selectedData'),
        dash.dependencies.Input('date-picker-range', 'start_date'),
        dash.dependencies.Input('date-picker-range', 'end_date')
    ]
)
def update_scatter_plot(map_data, start_date, end_date):
    states = [each['location'] for each in map_data['points']] if map_data else None # extract states
    data = category_month_scatter_plot(df, states, start_date, end_date)
    return {
        'data': data,
        'layout': go.Layout(
            margin={'t': 10, 'r': 10,}
        )
    }


@app.callback(
    dash.dependencies.Output('category-bar-plot', 'figure'),
    [
        dash.dependencies.Input('map-plot', 'selectedData'),
        dash.dependencies.Input('date-picker-range', 'start_date'),
        dash.dependencies.Input('date-picker-range', 'end_date')
    ]
)
def update_bar_plot(map_data, start_date, end_date):

    states = [each['location'] for each in map_data['points']] if map_data else None # extract states
    data = category_bar_plot(df, states, start_date, end_date)

    return {
        'data': data,
        'layout': go.Layout(
            margin={'t': 10, 'r': 10,}
        )
    }


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Conclusiones

En este caso, creamos un tablero interactivo que permite a los usuarios comerciales obtener información sobre el rendimiento de las ventas y las ganancias por categoría en diferentes estados. Este tablero nos permitió profundizar en cada estado a través de la selección en un mapa y facilitó enormemente la comunicación de nuestros hallazgos a una audiencia no técnica.

## Consideraciones finales

En este caso, aprendió y armó una aplicación Dash simple con las siguientes características:

1. Componentes de diseño de la aplicación, como mapas y diagramas, usando Dash HTML y los componentes principales de Dash
2. Una fuente de datos `pandas` para esos componentes
4. Capacidades de entrada del usuario
5. Funciones de devolución de llamada que manipularon automáticamente los componentes en función de la entrada del usuario

Luego, este tablero se puede implementar en un servidor de AWS o en la intranet privada de una empresa para que lo usen todas las partes interesadas. En el siguiente caso, crearemos una aplicación Dash independiente y la ejecutaremos como un servicio web. También veremos otras funciones de devolución de llamada y diferentes tipos de gráficos, y usaremos una base de datos relacional como nuestra fuente de datos.